## Bahan untuk Code

# Data Preparation

`E-Commerce Product Classification` ini bertujuan untuk memprediksi kategori produk yang paling sesuai dengan setiap produk yang diunggah ke platform e-commerce. Dalam hal ini, kita menghadapi tantangan untuk memastikan bahwa produk diklasifikasikan ke dalam kategori yang benar-benar sesuai dengan fitur-fiturnya.

Dalam bisnis e-commerce, klasifikasi produk yang akurat sangat penting untuk meningkatkan pengalaman pengguna dan efektivitas operasional. Salah satu perhatian utama adalah memastikan bahwa setiap produk diklasifikasikan ke dalam kategori yang paling relevan. Oleh karena itu, kita lebih mengutamakan metrik bisnis yang mirip dengan yang digunakan dalam prediksi churn, yaitu Recall.

Dalam konteks ini, metrik bisnis Recall berfokus pada kemampuan model untuk mengidentifikasi dan mengklasifikasikan produk ke dalam kategori yang sebenarnya sesuai, bahkan jika produk memiliki fitur yang kompleks atau karakteristik yang ambigu. Dengan mengutamakan Recall, kita berusaha untuk meminimalkan kesalahan prediksi yang mengarah pada produk yang seharusnya termasuk dalam suatu kategori namun salah diklasifikasikan.

Tentu saja, ada trade-off yang perlu diperhatikan, yaitu kemungkinan meningkatnya jumlah False Positive (produk yang salah diklasifikasikan ke dalam kategori tertentu). Namun, dalam konteks ini, kesalahan ini dianggap lebih dapat diterima daripada kesalahan False Negative, di mana produk yang seharusnya termasuk dalam suatu kategori tidak terklasifikasikan dengan benar.

Dengan mengutamakan metrik bisnis Recall dalam `E-Commerce Product Classification` ini, kita berharap untuk meningkatkan akurasi dalam mengkategorikan produk, yang pada gilirannya akan meningkatkan pengalaman pengguna dan efisiensi operasional bisnis e-commerce secara keseluruhan. Metrik ini membantu kita mendeteksi dan mencegah produk yang benar-benar sesuai dengan suatu kategori (kasus positif) lebih efisien daripada mengkhawatirkan tentang produk yang salah diklasifikasikan ke dalam kategori tertentu (kasus negatif)."

# Importasi Library

In [1]:
#Import library untuk data preparation dan visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import warnings for ignore the warnings
import warnings 
warnings.filterwarnings("ignore")

# import pickle and json file for columns and model file
import pickle
import json
import joblib
import copy

# Import train test split untuk splitting data
from sklearn.model_selection import train_test_split
import yaml
from tqdm import tqdm
import os

In [2]:
ecommerce_data = pd.read_csv("dataset/1 - raw data/training_data.csv")

In [3]:
ecommerce_data

,title,category
0,The Three Amigos,None
1,Home Essentials Blue Floral Glass Vintage Styl...,Home & Kitchen
2,Cooper Wiring Quiet Toggle Switch Single Pole ...,Tools & Home Improvement
3,Baseboarders&reg; Wall Brackets,Tools & Home Improvement
4,The Great Wave Off Kanagawa Custom Rectangle M...,Office Products
...,...,...
20183,D0309-WML Leviton Decora Screwless Snap-On Wal...,None
20184,South Shore Axess Small 2-Door Storage Cabinet...,None
20185,Patriotic Flip Flop Ornament Assortment of 2,Home & Kitchen
20186,Cybrtrayd Life of the Party A116 Horse Chocola...,None


# Data Definition

Data yang digunakan dalam analisis ini mencakup informasi tentang nama produk dan kategori produk yang diambil dari dataset berikut:

Sumber Data: [Predict Categories of Items using NLP](https://www.kaggle.com/datasets/shivam1298/predict-categories-of-items-using-nlp)

Dataset ini terdiri dari 20.188 baris data dengan dua kolom utama:

1. **title**: Kolom ini berisi nama produk yang dijual di platform e-commerce.
2. **category**: Kolom ini berisi kategori atau klasifikasi produk yang sesuai dengan produk tersebut.

# Data Validation

In [4]:
#Check informasi data
ecommerce_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20188 entries, 0 to 20187
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     20188 non-null  object
 1   category  20188 non-null  object
dtypes: object(2)
memory usage: 315.6+ KB


In [5]:
#Menghitung nilai null pada kolom
ecommerce_data.isnull().sum()

title       0
category    0
dtype: int64

In [6]:
#Check data duplicate
ecommerce_data.duplicated().sum()

0

In [7]:
## Cek masing masing proporsi data
ecommerce_data['category'].value_counts()

None                        10123
Home & Kitchen               4960
Tools & Home Improvement     2080
Office Products              1144
Grocery & Gourmet Food       1102
Industrial & Scientific       588
Electronics                   191
Name: category, dtype: int64

Menghilangkan baris pada label None karena setelah ditelusuri memang tidak memiliki kategori yang spesifik dan justru mirip dengan label yang sudah ada

In [8]:
## drop baris dengan label None
ecommerce_data.drop(ecommerce_data[ecommerce_data.category=='None'].index, inplace=True)

# Data Splitting
Melakukan spliting data untuk memisahkan set training, set validation dan set test yang terbagi atas variabel x dan y

In [9]:
ecommerce_data

,title,category
1,Home Essentials Blue Floral Glass Vintage Styl...,Home & Kitchen
2,Cooper Wiring Quiet Toggle Switch Single Pole ...,Tools & Home Improvement
3,Baseboarders&reg; Wall Brackets,Tools & Home Improvement
4,The Great Wave Off Kanagawa Custom Rectangle M...,Office Products
5,Nemcor Pittsburgh Penguins NHL #1 Ultimate Fan...,Home & Kitchen
...,...,...
20175,Curtain Chic Supple Microfiber Cumberbund Vala...,Home & Kitchen
20176,Jack &amp; Jason's Pancake &amp; Waffle Mix - ...,Grocery & Gourmet Food
20178,4 Swivel Casters 4&quot; Rubber Double Brake B...,Industrial & Scientific
20185,Patriotic Flip Flop Ornament Assortment of 2,Home & Kitchen


In [10]:
ecommerce_data.shape

(10065, 2)

In [11]:
# Pemisahan Variabel X dan Y
X = ecommerce_data.drop(columns = "category")
y = ecommerce_data["category"]

In [12]:
X 

,title
1,Home Essentials Blue Floral Glass Vintage Styl...
2,Cooper Wiring Quiet Toggle Switch Single Pole ...
3,Baseboarders&reg; Wall Brackets
4,The Great Wave Off Kanagawa Custom Rectangle M...
5,Nemcor Pittsburgh Penguins NHL #1 Ultimate Fan...
...,...
20175,Curtain Chic Supple Microfiber Cumberbund Vala...
20176,Jack &amp; Jason's Pancake &amp; Waffle Mix - ...
20178,4 Swivel Casters 4&quot; Rubber Double Brake B...
20185,Patriotic Flip Flop Ornament Assortment of 2


In [13]:
y

1                  Home & Kitchen
2        Tools & Home Improvement
3        Tools & Home Improvement
4                 Office Products
5                  Home & Kitchen
                   ...           
20175              Home & Kitchen
20176      Grocery & Gourmet Food
20178     Industrial & Scientific
20185              Home & Kitchen
20187    Tools & Home Improvement
Name: category, Length: 10065, dtype: object

In [14]:
#Split Data 80% training 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state = 123)

In [15]:
X_train

,title
5544,Imax 84532 Tenor Large Recycled Glass Vase
3120,Darice 6202-113 10-Light 5-Point Star Tree Top...
1387,5 Piece Full Size Frozen Bedding Set Includes ...
1075,New Star Foodservice Knives (Set of 12)
13665,Mr &amp; Mrs T Bold &amp; Spicy Bloody Mary Mi...
...,...
19690,"Aviditi MD24126 Multi-Depth Corrugated Box, 24..."
15580,Perfume Studio&reg; Graduated Glass Dropper Bo...
10534,"Brother HL-L2340DW Compact Laser Printer, Mono..."
2793,Hello Kitty Wastebasket - Garbage Can


In [16]:
X_test

,title
11527,Dallmayr Dyawa Antara Ground Coffee Gift Box 5...
16233,"Enclosures, Boxes, &amp; Cases 3.15 x 1.58 x 0..."
742,Custom Size Runner Black Solid Single Color Pl...
9797,AT&amp;T EL52210 2-Handset Cordless Answering ...
7806,36&quot;w X 24&quot;h Steel Return Air Grilles...
...,...
15079,Santa Fe Rx Dehumidifier 16 X 20 X 2&quot; Mer...
7689,6-32 x 1/4 Ivory Switch Plate Screw (25 pieces)
9702,WGI Love Table Top Sign Decor
16814,NMC HB13 LOCKOUT TAGOUT AN OPEN AND SHUT CASE ...


In [17]:
y_train

5544              Home & Kitchen
3120              Home & Kitchen
1387              Home & Kitchen
1075              Home & Kitchen
13665     Grocery & Gourmet Food
                  ...           
19690            Office Products
15580    Industrial & Scientific
10534            Office Products
2793              Home & Kitchen
7250              Home & Kitchen
Name: category, Length: 8052, dtype: object

In [18]:
y_test

11527      Grocery & Gourmet Food
16233     Industrial & Scientific
742                Home & Kitchen
9797              Office Products
7806     Tools & Home Improvement
                   ...           
15079    Tools & Home Improvement
7689     Tools & Home Improvement
9702               Home & Kitchen
16814     Industrial & Scientific
751               Office Products
Name: category, Length: 2013, dtype: object

In [19]:
# Split data train menjadi train dan validation set
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, 
                                                    test_size=0.4, 
                                                    random_state=42,
                                                    stratify = y_test
                                                   )

In [20]:
X_test

,title
16393,Milwaukee 8986-20 Variable Temperature Heat Gun
11232,Glen Lochey - 4 Ounce
7152,"Juvale Faux, Folding, Wooden, Leather, Storage..."
19231,BirthdayExpress Pocoyo Party Supplies - Revers...
17346,1/8&quot; x 25' White Shock Bungee Rubber Rope...
...,...
127,"Winsome Wood 3-Tier Bookcase, Espresso"
18337,"Maxwell House Iced Coffee Concentrate, Vanilla..."
14312,"4EVER Candle Red Apple, Wedding Birthday Chris..."
12403,Warrior Flexible WP-9FV SR-9FV TIG Welding Tor...


In [21]:
X_valid

,title
1867,Large Barber Shop w/Pole Neon Sign
14588,Hortense B. Hewitt Cake Top Wedding Accessorie...
18708,"Golden State Art, Cream Photo Wood Collage Fra..."
8865,Marvel Ultimate Spider-Man 3-Piece Lunch Box Set
9777,Reusable Sandwich and Snack Bag (3 Piece Set) ...
...,...
9323,AIRDODO European Decorative Style 70-inch By 9...
13425,Refillable Grill lighter (Handi-Hand)
9967,Attack Plaque Magnet T-Rex
2174,"The Sharper Image 3-Piece Multitool Kit, Blue"


In [22]:
y_test

16393    Tools & Home Improvement
11232      Grocery & Gourmet Food
7152               Home & Kitchen
19231              Home & Kitchen
17346    Tools & Home Improvement
                   ...           
127                Home & Kitchen
18337      Grocery & Gourmet Food
14312              Home & Kitchen
12403    Tools & Home Improvement
8794     Tools & Home Improvement
Name: category, Length: 1207, dtype: object

In [23]:
y_valid

1867     Tools & Home Improvement
14588      Grocery & Gourmet Food
18708              Home & Kitchen
8865               Home & Kitchen
9777               Home & Kitchen
                   ...           
9323               Home & Kitchen
13425              Home & Kitchen
9967               Home & Kitchen
2174     Tools & Home Improvement
4244               Home & Kitchen
Name: category, Length: 806, dtype: object

In [24]:
joblib.dump(X_train, "C:\\Users\\hp\\Portofolio Data Science\\6 - E-Commerce Product Classification\\dataset\\2 - processed\\X_train.pkl")
joblib.dump(y_train, "C:\\Users\\hp\\Portofolio Data Science\\6 - E-Commerce Product Classification\\dataset\\2 - processed\\y_train.pkl")
joblib.dump(X_valid, "C:\\Users\\hp\\Portofolio Data Science\\6 - E-Commerce Product Classification\\dataset\\2 - processed\\X_valid.pkl")
joblib.dump(y_valid, "C:\\Users\\hp\\Portofolio Data Science\\6 - E-Commerce Product Classification\\dataset\\2 - processed\\y_valid.pkl")
joblib.dump(X_test, "C:\\Users\\hp\\Portofolio Data Science\\6 - E-Commerce Product Classification\\dataset\\2 - processed\\X_test.pkl")
joblib.dump(y_test, "C:\\Users\\hp\\Portofolio Data Science\\6 - E-Commerce Product Classification\\dataset\\2 - processed\\y_test.pkl")
#joblib.dump(train_set, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\train_set.pkl") 

['C:\\Users\\hp\\Portofolio Data Science\\6 - E-Commerce Product Classification\\dataset\\2 - processed\\y_test.pkl']